<img src="Branded_Logo_CUDenver.PNG" width="150">

## <center>CSCI 4580/5580 - Data Science – Spring 2022</center>
<center>Assignment 3: KNN, Naive Bayes and Text Feature Selection</center><center><font color='red'>Deadline: May 6, 2022 - 11:59 PM</font></center><center>Total Points: 100</center>

## Instructions
- Please note that this assignment must be done individually. By submitting this assignment, you certify that this is your own work, your code will be checked against other submissions and resources using automatic tools. Everyone should be getting a hands on experience in this course. You are free to discuss course material with fellow students, and we encourage you to use Internet resources to aid your understanding, but the work you turn in, including all code and answers, must be your own work.

## Deliverables
You need to submit a single .ipynb file on Canvas, named your-lastname_your-first-name.ipynb. For example, if your name is John Smith, you should name the file smith_john.ipynb.
- Please do not include extra files such as the input datasets in your submission.
- Answer Questions 1 - 7 in the designated cells. Please do not add or remove any cells. 
- Please download your submission file after submission and make sure it is not corrupted. Use the 'Run All' option from the 'Cell' menu to ensure all cells run without any issues. We will not be responsible for corrupted submissions and will not take a resubmission after the deadline.

## Need Help?
If you need help with this lab, please email me at sundous.hussein@ucdenver.edu or come to my office hours. We also encourage you to ask your questions on the designated channel for the assignment on Microsoft Teams. This way, you may receive assistance from your classmates that might’ve ran through the same issues.

Let's Get Started!
===
## Overview

Supervised Learning is the process of learning based on known examples which have both an input and a labeled output. Using the data and labels provided in the training dataset, models can learn a prediction function or mapping, which may then be applied to unseen test data. As we discussed in the class, some common methods for supervised learning include the K-Nearest-Neighbors Classifier (KNN) and Naive Bayes.

### Supervised Learning on Text Data

For this assignment, we'll explore kNN and Naive Bayes further and look at different feature selection strategies. First of all, download the dataset for this assignment from Canvas and unpack into an "Assignment 3" directory. 

The dataset is from the RCV1 v2 corpus which is a collection of news articles and category labels. There are 103 categories, and each document can lie in one or more categories. 

The sparse train and test matrices are encoded as nnz x 3 integer matrices, where nnz is the total number of words in all docs. Each row of these matrices contains (doc, word, count) triples. We will store those in sparse matrices whose dimensions are ndocs x nwords

### Application

Your assignment is to perform supervised learning using both the KNN and Naive Bayes classifiers, and evaluate the results. You will also perform multiple types of text feature selection to choose subsets of the total features to use for classification, and evaluate these results as well.

In [1]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

Now read the data files. 

In [2]:
iwords = np.loadtxt("data/words.imat.txt",dtype=np.int32,)          # training data matrix in nnz x 3 form - rows are (doc, word, count) triples
cats = np.loadtxt("data/cats.imat.txt",dtype=np.int32)             # training labels in an ndocs x ncats matrix
tiwords = np.loadtxt("data/testwords.imat.txt",dtype=np.int32)     # test data matrix in nnz x 3 form
tcats = np.loadtxt("data/testcats.imat.txt",dtype=np.int32)        # testing labels in an ndocs x ncats matrix

iwords and tiwords encode the sparse train and test matrices. Each row of these matrices contains (doc, word, count) triples. We will store those in sparse matrices whose dimensions are ndocs x nwords. The exact matrix type is CSR which stands for compressed sparse rows. Look up its description here: https://en.wikipedia.org/wiki/Sparse_matrix

In [3]:
fwords = iwords[:,2]
train = sp.csr_matrix((fwords, (iwords[:,1], iwords[:,0])))
ndocs = train.shape[0]
nwords = train.shape[1]

test = sp.csr_matrix((tiwords[:,2], (tiwords[:,1], tiwords[:,0])),shape=(4000,nwords))  # need to match the number of cols (words)
ntdocs = test.shape[0]

Although the dataset has 103 category labels, we'll focus on training a single category. We will use Category 6, which is fairly evenly balanced between positives and negatives. 

In [4]:
cat6 = cats[:,6]
tcat6 = tcats[:,6]

# Part 1: KNN Classification

This time we'll train Scikit-Learn's builtin kNN classifier. 

In [5]:
from sklearn.neighbors import KNeighborsClassifier
knnc = KNeighborsClassifier(n_neighbors=3)

And then "fit" it to the data. Notice how fast this is! Of course, training kNN is basically a no-op, and predicting is where all the work lies:

In [6]:
knnc.fit(train, cat6)

KNeighborsClassifier(n_neighbors=3)

Now let's try predicting. This will actually take some time, but hopefully only a few tens of seconds...

In [7]:
preds = knnc.predict(test);preds

array([0, 1, 1, ..., 0, 0, 0])

Now we can compute the accuracy on the actual test set labels. 

In [8]:
np.mean(tcat6 == preds)

0.82225

Not bad, but kNN is very sensitive to the distance function as we discussed in class. Let's try first normalizing each row of "train" and "test" by the L2-norm of that row, i.e. we will divide by the square root of the sum of the squared word counts in the row. Comparing the normalized documents should do much better. 

Remember, the L2-Norm for a vector $x$ can be written as:

$$  \lVert x \rVert_2  = \sqrt{x_1^2 + x_1^2 + \cdots + x_n^2}$$

## Q1 [20 points]: 

Create two new variables `trainnmat` and `testnmat` which contain the L2-normalized versions of `train` and `test` respectively.

L2-normalization should be performed across each row $r$ of the `train` and `test` matrices, so that:

$$ \sqrt{r_1^2 + r_2^2 + \cdots + r_n^2} = 1 $$

However, you must compute the L2-normalized matrices using `numpy` and `scipy.sparse` operations, using the scikit-learn `normalize` function to generate your answer is not valid.
Also ensure that your output matrix is still sparse. Many NumPy operations will either fail or convert CSR matrices to dense matrices. We prefer to keep this matrix in CSR format for the duration of this lab.

The documentation for the following sparse matrix function may be useful for you:

- `scipy.sparse.csr_matrix` [Docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)
- `scipy.sparse.csr_matrix.sum` [Docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.sum.html)
- `scipy.sparse.csr_matrix.power` [Docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.power.html#scipy.sparse.csr_matrix.power)
- `scipy.sparse.spdiags` [Docs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.spdiags.html)

Above we have already imported `scipy.sparse` as `sp`.

**HINT:** For both `train` and `test` you need to structure your normalization as a matrix multiply operation. Start by creating a diagonal matrix where the squared sums of each row are along the diagonal elements.


In [9]:
trainnmat =sp.spdiags(np.transpose(1/np.sqrt(np.max(train.multiply(train).sum(1),1))),0,ndocs,ndocs)*train
testnmat =sp.spdiags( np.transpose(1 / np.sqrt(np.max(test.multiply(test).sum(1),1))),0,ntdocs,ntdocs)*test

In [10]:
### TEST CASES - DO NOT MODIFY
# These test cases check that your normalization is close (within numerical precision) of the answer
# from sklearn's normalize function.
from sklearn.preprocessing import normalize
skl_norm_train = normalize(train)
skl_norm_train.sort_indices()

skl_norm_test = normalize(test)
skl_norm_test.sort_indices()

trainnmat.sort_indices()
testnmat.sort_indices()

assert(np.allclose(trainnmat.data,skl_norm_train.data))
assert(np.allclose(testnmat.data,skl_norm_test.data))

Now train and evaluate a knn model using the normalized datasets:

In [11]:
knnc.fit(trainnmat, cat6)
preds = knnc.predict(testnmat);preds
np.mean(tcat6 == preds)     

0.90275

## Q2 [10 points]: 
What is the accuracy of your kNN classifier on the normalized data?

In [12]:
# Add your answer to Question 2 here
# The accuracy of my KNN classifier on the normalized data came out to be 0.90275.

# Part 2: Naive Bayes Classification

Let's create a Multinomial Naive Bayes classifier. The multinomial distribution models each word in a document as being drawn from a category-specific probability distribution over the vocabulary. 

In [13]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

Now we can fit the NB model, which simply means tabulating the word counts for each label.

In [14]:
mnb.fit(train, cat6)

MultinomialNB()

...and predict the category labels for cat6:

In [15]:
preds = mnb.predict(test);preds

array([0, 1, 1, ..., 1, 0, 1])

## Q3 [10 points]: 

Let's check the accuracy below. How did NB do compared to kNN? 

In [16]:
np.mean(tcat6 == preds)

0.8975

In [17]:
# Add your answer to Question 3 here
# The accurary for the NB came out to be 0.8975 meaning that the KNN performed better
# than the NBB.

# Part 3: Frequency-Based Feature Selection

We argued that frequency-based feature selection is useful approach to culling features to reduce the size of a model. In some cases, accuracy improves as well. 

Let's construct a matrix rcounts which contains the counts of features from the training set. 

In [18]:
rcounts = (train>0).sum(0)
rcounts

matrix([[3728, 4438, 3488, ...,    1,    1,    1]], dtype=int32)

In [19]:
rcounts.size

31331

Next we'll find the indices of non-zero elements of the predicate (rcounts >= count). These are the features that occurred at least count times. ii is the indices of these features.

We then extract the columns of the train and test matrices corresponding to those high-count features. The last line tells us how many features we kept.

In [20]:
threshold = 2
ii0 = np.asarray(np.nonzero(rcounts >= threshold)[1])
ii = ii0.reshape(ii0.size,)
strain = train[:,ii]
stest = test[:,ii]
ii.size

18330

Finally we run training and evaluation on train/test data restricted to those columns to get an accuracy score.

In [21]:
mnb.fit(strain, cat6)
preds = mnb.predict(stest);preds
np.mean(tcat6 == preds)

0.8955

## Q4 [20 points]
For the following question, use the `threshold` variable set above.

For threshold counts of 1, 2, 5, 10, 20, tabulate the count, number of features kept, and the accuracy score. 

In [22]:
# Add your answer to Question 4 here

threshold_counts = [1,2,5,10,20]
data = []
for index in threshold_counts:
    ii0 = np.asarray(np.nonzero(rcounts >= index)[1])
    ii = ii0.reshape(ii0.size,)
    strain = train[:,ii]
    stest = test[:,ii]
    mnb.fit(strain, cat6)
    preds = mnb.predict(stest)
    data.append([index, ii.size, np.mean(tcat6 == preds)])

data_1 = pd.DataFrame(data,columns=['Threshold','No of features', 'Accuracy'])

print(data_1)
print("Original model size = " ,rcounts.size)
print("Reduced model size = ", ii.size)
print("Difference in size = ", rcounts.size - ii.size)

   Threshold  No of features  Accuracy
0          1           31331   0.89750
1          2           18330   0.89550
2          5            9632   0.89000
3         10            6169   0.88675
4         20            4046   0.88400
Original model size =  31331
Reduced model size =  4046
Difference in size =  27285


## Q5 [10 points]

How did frequency-based feature selection affect accuracy? How much was the model size (proportional to words kept) reduced? 

(`ii.size` is the reduced model size, `rcounts.size` is the original model size.)

In [23]:
# Add your answer to Question 5 here
# From the above questions dataframe it can be seen that with the increase in the
# no of features the accuracy is increasing. In the above question, I have shown the 
# original model size =31331 and reduced model size was 4046 . So the difference is
# 27,285. The reduced model size after the threshold variable after extracting the 
# columns of the train and test matrices corresponding to those high-count features
# was 18330. 

# Part 4: Feature Selection Based on Statistical Tests

Let's try a more sophisticated method to select the (word) features. Since we have count data, the Chi-Squared test is appropriate. Remember that Chi-squared can be used on 2x2 contingency tables to determine if there is an "interaction". In this case, we test if the label interacts with the feature. 

In [24]:
fcounts = np.zeros((4,nwords))           # holds the counts of various combinations of label and feature
expected = np.zeros((4,nwords))          # holds the predicted counts of those same combinations

In [25]:
mcat6 = cat6.reshape(1,ndocs)
fcounts[0,:] = mcat6 * (train > 0)                                   # label and word both = 1
fcounts[1,:] = (1-mcat6) * (train > 0)                               # label = 0, word = 1
fcounts[2,:] = np.sum(mcat6) * np.ones((1,nwords)) - fcounts[0,:]    # label = 1, word = 0
fcounts[3,:] = np.sum(1-mcat6) * np.ones((1,nwords)) - fcounts[1,:]  # label = 0, word = 0

Review the definition of the CHI-squared test here:
https://en.wikipedia.org/wiki/Chi-squared_test .

In [26]:
allf = np.sum(fcounts,axis=0)
pword0 = (fcounts[2,:] + fcounts[3,:]) / allf    # prob word = 0
pword1 = (fcounts[0,:] + fcounts[1,:]) / allf    # prob word = 1
pcat0 = (fcounts[1,:] + fcounts[3,:]) / allf     # prob label = 0
pcat1 = (fcounts[0,:] + fcounts[2,:]) / allf     # prob label = 1

If label and word occurence are independent, then the expected count should be the product of the total count and the two corresponding probabilities:



## Q6 [20 points]: 

Implement the expected counts here. 

**HINT** This can be computed entirely with the variables from the previous code cell, with no other variables needed.

In [27]:
#TODO: Implement this
expected[0,:] = allf * pcat1 * pword1   # expected count with label = 1, word = 1
expected[1,:] = allf * pcat0 * pword1  # expected count with label = 0, word = 1
expected[2,:] = allf * pcat1 * pword0   # expected count with label = 1, word = 0
expected[3,:] = allf * pcat0 * pword0   # expected count with label = 0, word = 0

To compute the chi-squared statistic value, we want the sum of the squared differences between actual and predicted counts, normalized by the expected counts. The following two cells perform that calculation. We have to guard against divide-by-zero so we use a small lower-bound on the denominator.

In [28]:
diff = fcounts - expected
safeexpected = np.maximum(expected,1e-7)

In [29]:
chisquaredstat = np.sum(np.multiply(diff, diff) / safeexpected, axis=0)
chisquaredstat.shape

(31331,)

In [30]:
chisquaredstat

array([ 83.64630905,  97.8608861 , 660.13078932, ...,   0.8417048 ,
         0.8417048 ,   0.8417048 ])

We can then use the chi-squared statistic to choose the best features. The CHI-squared stastistic is a measure of interaction between a word feature and the label. The higher its value, the better that word should be as a predictor of the category. We set a threshold and take the words whose chi-squared statistic is above this value:

In [31]:
chisqthreshold = 0.001
ii0 = np.asarray(np.nonzero(chisquaredstat > chisqthreshold)[0])
ii = ii0.reshape(ii0.size,);ii.shape

(31281,)

Finally we use the indices of good features selected by the chi-squared filter to train and evaluate the Naive Bayes model.

In [32]:
strain = train[:,ii]
stest = test[:,ii]
mnb.fit(strain, cat6)
preds = mnb.predict(stest);preds
np.mean(tcat6 == preds)

0.898

## Q7 [10 points]

Using a chi-sqaured critical value table (one can be found here: [Link](https://people.richland.edu/james/lecture/m170/tbl-chi.html)), find the table row for 1 degree of freedom (`df=1`), and set `chisqthreshold` to a few different values from the table. Record your accuracies here.

In a few sentences describe how well this feature selection process works on the test data.

In [33]:
# Add your answer to Question 7 here
#using the chisqthreshold from the given link with df=1 

chisqthreshold = [0.001,0.004,0.016,2.706,3.841,5.024,6.635,7.879]
data_2 =[]
for index in chisqthreshold:
    ii0 = np.asarray(np.nonzero(chisquaredstat > index)[0])
    ii = ii0.reshape(ii0.size,)
    strain = train[:,ii]
    stest = test[:,ii]
    mnb.fit(strain, cat6)
    preds = mnb.predict(stest)
    data_2.append([index,np.mean(tcat6 == preds)])
    
data_3 = pd.DataFrame(data_2,columns=['Chisqthreshold','Accuracy'])
print(data_3)
print("The accuracy of the model decreased with the increase in the chisqthreshold values.")

   Chisqthreshold  Accuracy
0           0.001   0.89800
1           0.004   0.89750
2           0.016   0.89750
3           2.706   0.88300
4           3.841   0.88325
5           5.024   0.87925
6           6.635   0.87650
7           7.879   0.87425
The accuracy of the model decreased with the increase in the chisqthreshold values.
